# Python SDK configuration for _waylay-io_

This notebook illustrates how to connect to the Waylay Platform using the Python SDK (waylay-py) **on the `waylay-io` platform**.

The steps are the same as in [Authentication for the Enterprise platform](Authentication.ipynb), except that the 
_authentication provider_ endpoint is 

> `https://accounts-api-io.waylay.io`


In [1]:
from waylay import WaylayConfig, WaylayClient

## Authentication with stored profiles

The Python SDK connection settings can be stored in local profile files.

For this demo, we first delete the 'demo' profile (if it exists)

In [2]:
WaylayConfig.delete('demo-io')

waylay configuration removed: /Users/thomas/Library/Application Support/Waylay/python_sdk/.profile.demo-io.json


'/Users/thomas/Library/Application Support/Waylay/python_sdk/.profile.demo-io.json'

You can authenticate with a profile as follows. When the profile is not found on the file system, it will be created interactively

In [4]:
waylay_client = WaylayClient.from_profile('demo-io')

Authenticating to the Waylay Platform
Using authentication provider at [https://accounts-api.waylay.io]
> alternative endpoint (press enter to continue)?: https://accounts-api-io.waylay.io
Authenticating at '@waylay/auth'
Please provide client credentials for the waylay data client.
> apiKey : 86724b383e6318ae2551de28
> apiSecret : ········
> Do you want to store these credentials with profile=demo-io? [Y]: Y
Credential configuration stored as 
	/Users/thomas/Library/Application Support/Waylay/python_sdk/.profile.demo-io.json
Please make sure this file is treated securely.
If compromised, _Revoke_ the api-key on the Waylay Console!


Once a profile exists, it will be used until you delete it.

In [5]:
waylay_client = WaylayClient.from_profile('demo-io')

A client will use the endpoint configurations defined in the tenant (global) settings.

In [6]:
waylay_client.list_root_urls()

{'analytics': 'https://ts-analytics-io.waylay.io',
 'byoml': 'https://byoml-io.waylay.io',
 'api': 'https://io-testing.waylay.io',
 'storage': 'https://storage-io.waylay.io',
 'etl': 'https://etl-export-io.waylay.io',
 'data': 'https://data-io.waylay.io'}

In [7]:
waylay_client.analytics.about.version()

'tsanalytics.server 0.20.3'

you can check connection profile by printing the config object

In [8]:
print(waylay_client.config)

{"credentials": {"type": "client_credentials", "api_key": "86724b383e6318ae2551de28", "api_secret": "********", "accounts_url": "https://accounts-api-io.waylay.io"}, "profile": "demo-io", "settings": {}}


## Managing config profiles

The _WaylayConfig_ class provides the following methods to interact with config profiles.

In [9]:
# list profiles
WaylayConfig.list_profiles()

# delete profiles
WaylayConfig.delete('demo-copy')

# load configuration from profile
config = WaylayConfig.load('demo-io')
waylay_client = WaylayClient(config)

# .. alternatively ...
WaylayClient.from_profile('demo-io')

# save (altered) configuration in a profile
config.profile = 'demo-copy'
config.save()
config

waylay configuration removed: /Users/thomas/Library/Application Support/Waylay/python_sdk/.profile.demo-copy.json


<WaylayConfig({"credentials": {"type": "client_credentials", "api_key": "86724b383e6318ae2551de28", "api_secret": "********", "accounts_url": "https://accounts-api-io.waylay.io"}, "profile": "demo-copy", "settings": {}})>

## Alternative authentication methods
Alternatively explicitely configure a Waylay Client with client credentials or waylay token. (e.g. reading it from you own-defined environment variables)

### Authenticate with client credentials

In [10]:
import os
api_key = os.getenv('WAYLAY_TEST_USER_ID')
api_secret = os.getenv('WAYLAY_TEST_USER_SECRET')
accounts_url = os.getenv('WAYLAY_TEST_ACCOUNTS_URL',  'https://accounts-api-staging.waylay.io')
waylay_client_from_cc = WaylayClient.from_client_credentials(api_key, api_secret, accounts_url)
str(waylay_client_from_cc.config)

'{"credentials": {"type": "client_credentials", "api_key": "ca8b5bed95e6f29aeaaa4a2b", "api_secret": "********", "accounts_url": "https://accounts-api-staging.waylay.io"}, "profile": "_default_", "settings": {}}'

### Authenticate with a waylay token

In [11]:
# temporary user token generated from the client credentials above
token = waylay_client.config.get_valid_token()

In [12]:
token.domain

'io-testing.waylay.io'

In [13]:
# string representation is the signed token
signed_token_string = str(token)

In [14]:
waylay_client = WaylayClient.from_token(
    signed_token_string, 
    accounts_url=waylay_client.config.accounts_url
)

In [15]:
str(waylay_client.config)

'{"credentials": {"type": "token", "token": "*********", "accounts_url": "https://accounts-api-io.waylay.io"}, "profile": "_default_", "settings": {}}'

Note that clients with a _token credential_ configuration will fail after the token has expired. 

Clients with _client credentials_ will renew the token automatically when it expires.

### saving a profile elsewhere
You can export and reconstruct configurations also manually:

In [16]:
demo_config = WaylayConfig.load('demo-io')

In [17]:
demo_config_json = demo_config.to_dict(obfuscate=False)
restored_config = WaylayConfig.from_dict(demo_config_json)
restored_config.to_dict()

{'credentials': {'type': 'client_credentials',
  'api_key': '86724b383e6318ae2551de28',
  'api_secret': '********',
  'accounts_url': 'https://accounts-api-io.waylay.io'},
 'profile': 'demo-io',
 'settings': {}}

In [18]:
restored_client = WaylayClient(restored_config)

In [19]:
restored_client.list_root_urls()

{'analytics': 'https://ts-analytics-io.waylay.io',
 'byoml': 'https://byoml-io.waylay.io',
 'api': 'https://io-testing.waylay.io',
 'storage': 'https://storage-io.waylay.io',
 'etl': 'https://etl-export-io.waylay.io',
 'data': 'https://data-io.waylay.io'}

In [20]:
demo_config_json = demo_config.to_dict(obfuscate=False)

WaylayClient(WaylayConfig.from_dict(demo_config_json))

<WaylayClient(services=[analytics,byoml,timeseries,api,storage,util,etl,data],config={"credentials": {"type": "client_credentials", "api_key": "86724b383e6318ae2551de28", "api_secret": "********", "accounts_url": "https://accounts-api-io.waylay.io"}, "profile": "demo-io", "settings": {}})>